## NAV Process Map
The ProcessMap class can be used to display an interactive Process Map of your data frame. Inspired by [bupaR Process Map](https://www.bupar.net/processmaps.html).

The Widget supports interactivity by moving nodes and hovering over nodes and edges for more detailed information. You can also filter the frequency of traces in order to display only a subset of data. Lastly, you can choose which labels the edges should display.


In [1]:
import os
from ntnu_process_mining import ProcessMap

from pandas import read_csv

file = read_csv(os.path.join("files", "sepsis.csv"))
# This is needed because the sepsis dataset has some rows where case_id = NaN
df = file.fillna(value={'case_id':"N/A"})

ProcessMap(df)

ProcessMap(value=[{'from': 'START', 'to': 'ER Registration', 'freq': 264, 'abs_freq': 264, 'perf': '0', 'perf_…